In [1]:
import requests
from bs4 import BeautifulSoup as bs
import lxml
import json
import constant
import ast
import re

In [2]:
def get_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
    r = requests.get(url, headers = headers)
    return r

In [14]:
 #r = get_html('https://spb.kassir.ru/bilety-na-koncert?sort=1')
 r = get_html('https://spb.kassir.ru/bilety-v-teatr?sort=1')

In [4]:
print(r)

<Response [200]>


In [7]:
def re_format_kudago_and_kassir(date):
    # обрезаем длинный формат даты события (кассир)
    if len(date) > 10:
        # '2021-04-28 20:00:00' -> '2021-04-28'
        date =  date[0:10]
    list_date = date.split('-')
    # удаление нуля в числе 
    number = int(list_date[2])
    re_format_date = str(number) + " " + constant.date_format_kudago[int(list_date[1])-1] + " " + list_date[0]
    return re_format_date

In [16]:
html = bs(r.text, 'html.parser')
event = []
soup = html.find_all('div', class_="col-xs-2")
event = []
for soup_event in soup[2:22]:
    json_string = soup_event.find('div', class_='event').attrs['data-ec-item']
    # пробуем получить словарь из json_string
    try: 
        first_part_event = json.loads(json_string)
    # в json_string возможно будут исключения типа "" -> удаляем их
    except:
        try:
            new_json_string = re.sub(r'[^:]""',' ', json_string).replace(' "', ' ').replace('" ', ' ')
            first_part_event = json.loads(new_json_string)
        except:
            continue
    # id события
    id_parse = first_part_event['id']
    # производим сравнение id каждого мероприятия с последним обработанным (записанным)
    # минимальная и максимальная стоимость события
    min_cost = first_part_event['minPrice']
    max_cost = first_part_event['maxPrice']
    # формируем строку стоимости
    if min_cost == max_cost:
        cost = str(max_cost) + ' рублей'
    else:
        cost = 'от ' + str(min_cost) + ' до ' + str(max_cost) + ' рублей'
    # дата события
    date = first_part_event['date']
    # если дата события не один день, то она хранится в виде словаря
    if type(date) == dict:
        date_start = re_format_kudago_and_kassir(first_part_event['date']['start_min'])
        date_stop = re_format_kudago_and_kassir(first_part_event['date']['start_max'])
        # если один день
    else:
        date_start = re_format_kudago_and_kassir(date)
        date_stop = None
        # категория
    type_event = first_part_event['category']

        # получаем вторую часть информации о событии в script: image, title, address, link 
    try:
        second_part_event = json.loads(str(soup_event.find('script', type="application/ld+json").string))
    except:
        continue
        # ссылка на изображения события
    img = second_part_event['image']
        # название события
    title = second_part_event['name']
    discounted = 0
        # адрес проведения события
    address_place = second_part_event['location']['address']
        # проверяем наличие годода Санк-Петербург в адресе
    if address_place.startswith('Санкт'):
            # если есть убираем
        address_spb = address_place.split(', ')
        address_place = ', '.join(address_spb[1:])
        # добавляем перед адресом название места проведения события
    address = second_part_event['location']['name'] + ", " + address_place
        # МЕТРО В РАЗРАБОТКЕ
    metro = None
        # ссылка на событие
    full_link = second_part_event['url']    

    event.append((id_parse, type_event, img, title, date_start, date_stop, cost, discounted, address, metro, full_link))
print(event)

[(8629, 'Театр', 'https://cdn.kassir.ru/spb/poster/e3/e3162241c153c1c8c39939d0935ed4a8.jpg', 'Айболит (Кукольный театр сказки)', '9 апреля 2021', '2 мая 2021', 'от 300 до 800 рублей', 0, 'Кукольный театр сказки, Московский пр., д. 121', None, 'https://spb.kassir.ru/teatr/aybolit-8629'), (798654, 'Театр', 'https://cdn.kassir.ru/spb/poster/08/08a3457197dda7a3ce7126ed5e946923.jpg', 'Краденое солнце', '9 апреля 2021', None, '450 рублей', 0, 'Театр кукол «Бродячая собачка», пр.Стачек 59', None, 'https://spb.kassir.ru/teatr/kradenoe-solntse#798654'), (793103, 'Театр', 'https://cdn.kassir.ru/spb/poster/13/138ce41e3d7303541f1c2d8e479a9d19.jpg', 'Турандот', '9 апреля 2021', None, 'от 1100 до 3500 рублей', 0, 'Театр Мастерская, ул. Народная, дом 1', None, 'https://spb.kassir.ru/teatr/turandot-teatr-masterskaya#793103'), (23297, 'Театр', 'https://cdn.kassir.ru/spb/poster/92/92cdf54f2fc9528bf35cf28054b3f41e.jpg', 'Джульетта выжила', '9 апреля 2021', '23 мая 2021', 'от 550 до 1000 рублей', 0, 'Теат

In [155]:
def re_format_kudago(date):
    if len(date) > 10:
        # '2021-04-28 20:00:00'
        date =  date[0:10]
    list_date = date.split('-')
# удаление нуля в числе 
    number = int(list_date[2])
    re_format_date = str(number) + " " + constant.date_format_kudago[int(list_date[1])-1] + " " + list_date[0]
    return re_format_date

In [150]:
x = '2021-04-28 20:00:00'
y =  x[0:10]
print(y)

2021-04-28


In [127]:
x = '{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}'


print(x)

{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}


In [128]:
w = re.sub(r'[^:]""',' ', x).replace(' "', ' ').replace('" ', ' ')
print(w)
y = x.replace('""', '"').replace(' "', ' ').replace('" ', ' ')


{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}


In [129]:
json.loads(w)

{'name': 'Гречанинов. Демественная литургия',
 'id': 778220,
 'price': 300,
 'category': 'Театр',
 'category_id': 3009,
 'image': 'https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg',
 'venueName': 'Государственная академическая Капелла',
 'position': 1,
 'cat1': 'Театр',
 'cat2': '',
 'eventId': 778220,
 'eventName': 'Гречанинов. Демественная литургия',
 'minPrice': 300,
 'maxPrice': 450,
 'date': '2021-03-31 19:00:00',
 'avail': 390,
 'placementId': 107}